Alexis Perumal, 5/4/20
HW22, Level 1 with help from Kevin Nguyen

In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"


In [0]:
import findspark
findspark.init()

# New Section

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-05-07 22:09:02--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  3.70MB/s    in 0.2s    

2020-05-07 22:09:02 (3.70 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


In [37]:
from pyspark import SparkFiles
# url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
category_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep="\t", header=True)
category_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [0]:
from pyspark.sql.types import IntegerType
category_df = category_df.withColumn("customer_id", category_df["customer_id"].cast(IntegerType()))
category_df = category_df.withColumn("product_parent", category_df["product_parent"].cast(IntegerType()))
category_df = category_df.withColumn("star_rating", category_df["star_rating"].cast(IntegerType()))
category_df = category_df.withColumn("helpful_votes", category_df["helpful_votes"].cast(IntegerType()))
category_df = category_df.withColumn("total_votes", category_df["total_votes"].cast(IntegerType()))

In [39]:
# Count the number of rows
category_df.count()

3105520

In [0]:
# For development purposes, limit the size of the dataframe so things go quicker!
# category_df = category_df.limit(1000)

In [41]:
# Count the number of rows
category_df.count()

3105520

In [42]:
# Drop null values
category_df = category_df.dropna()
category_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [43]:
# Count the number of rows
category_df.count()

3105360

In [0]:
# OOPS! Don't drop duplicates of the product ID here, do it specifically when building
#.      the table 2 df.
# # Drop duplicates
# category_df = category_df.dropDuplicates(subset=['product_id'])
# category_df.show()
# category_df.count()

Note the schema of the four tables:

CREATE TABLE review_id_table (
  review_id TEXT PRIMARY KEY NOT NULL,
  customer_id INTEGER,
  product_id TEXT,
  product_parent INTEGER,
  review_date DATE -- this should be in the formate yyyy-mm-dd
);

-- This table will contain only unique values
CREATE TABLE products (
  product_id TEXT PRIMARY KEY NOT NULL UNIQUE,
  product_title TEXT
);

-- Customer table for first data set
CREATE TABLE customers (
  customer_id INT PRIMARY KEY NOT NULL UNIQUE,
  customer_count INT
);

-- vine table
CREATE TABLE vine_table (
  review_id TEXT PRIMARY KEY,
  star_rating INTEGER,
  helpful_votes INTEGER,
  total_votes INTEGER,
  vine TEXT
);


In [45]:
# Build the Table 1, in PySpark: review_id_table

from pyspark.sql.functions import to_date
review_id_df = category_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RQ58W7SMO911M|   12076615|0385730586|     122662979| 2005-10-14|
|  RF6IUKMGL8SF|   12703090|0811828964|      56191234| 2005-10-14|
|R1DOSHH6AI622S|   12257412|1844161560|     253182049| 2005-10-14|
| RATOTLA3OF70O|   50732546|0373836635|     348672532| 2005-10-14|
|R1TNWRKIVHVYOV|   51964897|0262181533|     598678717| 2005-10-14|
|R2F53LI9KK9MOY|   31048862|0316769487|     862964341| 2005-10-14|
|R1KJ6MB7MRSQFF|   53000124|0805076069|     145341889| 2005-10-14|
|R2XIM9LT335WHE|   29732693|1581603681|     640542054| 2005-10-14|
|R1VE0FQQ0QTQJN|   48541186|0300108834|     915891133| 2005-10-14|
|R1VKEE2NWSWDRU|   15280864|0446531081|     880645124| 2005-10-14|
|R2UP6XSVYJBJ2H|   24209213|0976822105|     731931430| 2005-10-14|
|R21SYDQ70ILUC0|   52253037|1580085695|     586052746| 2005-10

In [46]:
# Build the Table 2, in PySpark: products table

# Extract products table info
products_df = category_df.select(["product_id", "product_title"]).dropDuplicates(subset=['product_id'])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0001714422|'C' Is for Clown ...|
|0001837192|Pieter Brueghel's...|
|0002005565|             Thieves|
|0002151731|The Greatest Bene...|
|0002216973|     Red Adam's Lady|
|0002217236|       Running Blind|
|000255111X|The Adventures of...|
|0004708237|Collins Gem Shubu...|
|0006544150|           In Xanadu|
|000717201X|The Lord of the R...|
|0007640617|Energize Your Lif...|
|0020710518|Mud Soldiers: Lif...|
|002075910X|Group Marriage;: ...|
|0020960107|The Only Way to C...|
|0023605103|Parables of Jesus...|
|0025239716|The Baseball Chro...|
|0025703811|Life on the Rim: ...|
|0027182223|The CHIMES OF ALY...|
|0027701808|             Dogsong|
|0028610881|Complete Idiot's ...|
+----------+--------------------+
only showing top 20 rows



In [47]:
products_df.count()

779717

In [48]:
# Build the Table 3, in PySpark: customers table
import pyspark.sql.functions as sf

customers_df = category_df.groupBy("customer_id").agg(sf.count("product_id").alias("customer_count"))
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   28735044|             9|
|   15607794|             1|
|   15596972|             1|
|   32371281|             2|
|   12084021|             1|
|   14462635|             1|
|   41052840|             5|
|   52068296|            94|
|   51739623|             1|
|   14544320|             1|
|   50093726|           151|
|   52925615|             1|
|   12266315|             1|
|   44917096|             1|
|   52833886|             1|
|   51854558|            57|
|   12404586|             2|
|   49850307|             8|
|   23478170|             1|
|   51088740|             1|
+-----------+--------------+
only showing top 20 rows



In [49]:
# Build the Table 4, in PySpark: vine_table

vine_table_df = category_df.select(["review_id", "star_rating", "helpful_votes", "total_votes",
                                       "vine"])
vine_table_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RQ58W7SMO911M|          4|            2|          3|   N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|
|R1DOSHH6AI622S|          4|            1|         22|   N|
| RATOTLA3OF70O|          5|            2|          2|   N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|
|R2XIM9LT335WHE|          4|            3|         10|   N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|
|R2UP6XSVYJBJ2H|          5|            0|          0|   N|
|R21SYDQ70ILUC0|          5|            1|          1|   N|
| R7M06Z88PD7SX|          4|           14|         14|   N|
| RRS38KZ4WB5O2|          5|            

Push to RDS

In [0]:
 # Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.ccavomop9z5f.us-east-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"postgres", 
          "password": "",
          "driver":"org.postgresql.Driver"}

In [0]:
# Table 1
# Write DataFrame to review_id_table table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [55]:
review_id_df.head()

Row(review_id='RQ58W7SMO911M', customer_id=12076615, product_id='0385730586', product_parent=122662979, review_date=datetime.date(2005, 10, 14))

In [0]:
# Table 2
# Write DataFrame to products table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [0]:
# Table 3 - Don't push this table since this is the second dataset.
# Write DataFrame to customers table in RDS
# customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [0]:
# Table 4
# Write DataFrame to vine_table table in RDS
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)